In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install comet_ml

In [ ]:
# used fo TPU
# Uncomment if you want to use TPU
# import collections
# from datetime import datetime, timedelta
# import os
# import requests
# import threading

# _VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
# VERSION = "xrt==1.15.0"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
# CONFIG = {
#     'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
#     'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
#         (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
# }[VERSION]
# DIST_BUCKET = 'gs://tpu-pytorch/wheels'
# TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
# TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
# TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# # Update TPU XRT version
# def update_server_xrt():
#   print('Updating server-side XRT to {} ...'.format(CONFIG.server))
#   url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
#       TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
#       XRT_VERSION=CONFIG.server,
#   )
#   print('Done updating server-side XRT: {}'.format(requests.post(url)))

# update = threading.Thread(target=update_server_xrt)
#update.start()

In [ ]:
import numpy as np
import os
from pathlib import Path
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
from pytorch_lightning import loggers

In [ ]:
class Discriminator(pl.LightningModule):
  def __init__(self, ndf):
      super().__init__()
      self.ndf = ndf

      self.condi = nn.Sequential(
            nn.Linear(in_features=10, out_features=32 * 32)
        )
      self.fc1 = nn.Sequential(
          nn.Conv2d(in_channels=2, out_channels=self.ndf, kernel_size=4, stride=2, padding=1, bias=False),  # 2 in channels because of condition
          nn.LeakyReLU(0.2)
      )
      self.fc2 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf, out_channels=self.ndf * 2, kernel_size=3, stride=2, padding=1, bias=False),
          nn.LeakyReLU(0.2)
      )
      self.fc3 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf * 2, out_channels=self.ndf * 3, kernel_size=3, stride=2, padding=1, bias=False),
          nn.LeakyReLU(0.2)
      )
      self.fc4 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf * 3, out_channels=self.ndf * 6, kernel_size=3, stride=2, padding=1, bias=False),
          torch.nn.Sigmoid()
      )
      self.fc5 = nn.Sequential(
          nn.Conv2d(in_channels=self.ndf * 6, out_channels=1, kernel_size=3, stride=2, padding=1,bias=False),
          nn.Sigmoid()
      )

  def forward(self, x, y):
    y = get_hot_vector_encode(y).detach()
    y = self.condi(y.view(-1, 10))
    y = y.view(-1, 1, 32, 32)
    
    x_cond = torch.cat((x, y), dim=1)

    x = self.fc1(x_cond)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    x = self.fc5(x)

    return x

In [ ]:
class Generator(pl.LightningModule):
  def __init__(self, latent_dim, ngf):
    super().__init__()
    self.ngf = 16
    self.n_features = latent_dim

    self.fc1 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=self.n_features + 10, out_channels=self.ngf * 8, kernel_size=4, stride=1, padding=0, bias=False),
        nn.LeakyReLU(0.2)
    )
    self.fc2 = nn.Sequential(
        nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, False),
        nn.LeakyReLU(0.2)
    )
    self.fc3 = nn.Sequential(
        nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, False),
        nn.LeakyReLU(0.2)
    )
    self.fc4 = nn.Sequential(
        nn.ConvTranspose2d(self.ngf * 2, self.ngf, 4, 2, 1, False),
          nn.LeakyReLU(0.2)
    )
    self.fc5 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=self.ngf, out_channels=1, kernel_size=3, stride=1, padding=1, bias=False),
          nn.Tanh()
    )

  def forward(self, x, y):
    y = get_hot_vector_encode(y).detach()
    x_cond = torch.cat((x, y), dim=1)

    x = self.fc1(x_cond)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    x = self.fc5(x)

    return x

In [ ]:
class DCGAN(pl.LightningModule):
  def __init__(self, hparams):
    super().__init__()
    self.hparams = hparams

    self.generator = Generator(hparams.latent_dim, hparams.ngf)
    self.discriminator = Discriminator(hparams.ndf)

    # cache for generated images
    self.generated_imgs = None
    self.last_imgs = None

    # For experience replay
    self.exp_replay = torch.tensor([])
    self.exp_replay_labels = torch.tensor([], dtype=torch.long)

  def binary_cross_entropy(self, y_hat, y):
    return F.binary_cross_entropy(y_hat, y)

  def forward(self, x, y):
    return self.generator(x, y)
  
  def training_step(self, batch, batch_nb, optimizer_idx):
    img, labels = batch
    self.last_imgs = img

    # Train Generator
    if optimizer_idx == 0:
      noise = torch.randn(img.shape[0], self.hparams.latent_dim, 1, 1)

      self.generated_img = self(noise, labels)

      valid_lbl = (0.8 - 1.1) * torch.rand(img.shape[0], 1) + 1.1  # soft labels

      g_loss = self.binary_cross_entropy(self.discriminator(self.generated_img, labels), valid_lbl)
      
      tqdm_dict = {'g_loss': g_loss}
      logs = {"g_loss": g_loss}
      return {"loss": g_loss, "log": logs, 'progress_bar': tqdm_dict}

    # Train Discriminator
    if optimizer_idx == 1:
      valid_lbl = (0.8 - 1.1) * torch.rand(img.shape[0], 1) + 1.1  # soft labels
      unvalid_lbl = (0.0 - 0.3) * torch.rand(img.shape[0], 1) + 0.3  # soft labels

      # Experience replay
      r_idx = max(1, self.hparams.experience_save_per_batch) # Getting the index
      self.exp_replay = torch.cat((self.exp_replay, self.generated_img[r_idx]), 0).detach() # Add our new example to the replay buffer
      
      self.exp_replay_labels = torch.cat((self.exp_replay_labels, labels[r_idx].unsqueeze(dim=0))).detach() # Add new labels to the replay buffer

      if self.exp_replay.size(0) >= self.hparams.experience_batch_size: # when we have enough example from the past train on them
         print("Experience replay epoch")
         
         self.exp_replay = self.exp_replay.unsqueeze(dim=1)
         unvalid_lbl = (0.0 - 0.3) * torch.rand(self.exp_replay.shape[0], 1) + 0.3  # soft labels
         fake_loss = self.binary_cross_entropy(self.discriminator(self.exp_replay, self.exp_replay_labels), unvalid_lbl)
         
         d_loss = fake_loss
         self.exp_replay = torch.tensor([])
         self.exp_replay_labels = torch.tensor([], dtype=torch.long)

      else:
        real_loss = self.binary_cross_entropy(self.discriminator(img, labels), valid_lbl)

        fake_loss = self.binary_cross_entropy(self.discriminator(self.generated_img.detach(), labels), unvalid_lbl)
        
        d_loss = (real_loss + fake_loss) / 2

      tqdm_dict = {'d_loss': d_loss}
      logs = {"d_loss": d_loss}
      return {"loss": d_loss, "log": logs, 'progress_bar': tqdm_dict}

  def configure_optimizers(self):
    opt_g = torch.optim.Adam(self.generator.parameters(), lr=self.hparams.lr, betas=(self.hparams.b1, self.hparams.b2))
    opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=self.hparams.lr, betas=(self.hparams.b1, self.hparams.b2))

    return opt_g, opt_d
    
  def train_dataloader(self):
    transform = transforms.Compose([transforms.Resize((32, 32)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5], [0.5])])
    dataset = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    return DataLoader(dataset, num_workers=self.hparams.num_workers, batch_size=self.hparams.batch_size)
    
  def on_epoch_end(self):
      noise = torch.randn(4, self.hparams.latent_dim, 1, 1)

      sample_img = self.generator(noise, torch.tensor((4, 6, 7, 1)))
      sample_img = sample_img.view(-1, 1, 32, 32)
      grid = torchvision.utils.make_grid(sample_img, nrow=2).permute(1, 2, 0)
      comet_logger.experiment.log_image(grid, f'generated_images_epoch{self.current_epoch}', step=self.current_epoch)

      if self.current_epoch % 5 == 0:
        trainer.save_checkpoint(checkpoint_folder + "/" + experiment_name + "_epoch_" + str(self.current_epoch) + ".ckpt")
        comet_logger.experiment.log_asset_folder(checkpoint_folder)

        if dirpath.exists() and dirpath.is_dir():
            shutil.rmtree(dirpath)

        # creating checkpoint folder
        access_rights = 0o755
        os.makedirs(checkpoint_folder, access_rights)

In [ ]:
def get_hot_vector_encode(labels):
    return torch.eye(10)[labels].view(-1, 10, 1, 1)

In [ ]:
# Parameters
experiment_name = "MNIST_CDCGAN_V1"
dataset_name = "MNIST"
checkpoint_folder = "CDCGAN/"
tags = ["CDCGAN", "MNIST"]
dirpath = Path(checkpoint_folder) # for saving checkpoints 

In [ ]:
# Hyperparameters
from argparse import Namespace

args = {
    'batch_size': 64,
    'lr': 0.0002,
    'b1': 0.5,
    'b2': 0.999,
    'latent_dim': 128,
    'level_of_noise': 0.1,
    'epochs': 100,
    'r_frequent': 100,
    'ndf': 16,
    'ngf': 16,
    'num_workers': 3,
    'experience_batch_size': 2000,
    'experience_save_per_batch': 1,
}

hparams = Namespace(**args)

In [ ]:
# init logger
comet_logger = loggers.CometLogger(
    api_key="",
    rest_api_key="",
    project_name="GAN",
    experiment_name=experiment_name,
)

#defining net
net = DCGAN(hparams)

#logging
comet_logger.experiment.set_model_graph(str(net))
comet_logger.experiment.add_tags(tags=tags)
comet_logger.experiment.log_dataset_info(dataset_name)

INFO:lightning:CometLogger will be initialized in online mode
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/luposx/gan/84f9a601dd6c4f6d9481ba83a8cd5970



In [ ]:
# deleting the checkpoint folder
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)
 
# creating checkpoint folder
access_rights = 0o755
os.makedirs(checkpoint_folder, access_rights)

In [ ]:
# Start training

trainer = pl.Trainer(# resume_from_checkpoint=checkpoint_folder + "/MNIST_GAN_V2_epoch2.ckpt",
                     logger=comet_logger,
                     max_epochs=args["epochs"]
                     )    
trainer.fit(net)
trainer.save_checkpoint(checkpoint_folder + "/" + experiment_name + "_epoch_" + str(args["epochs"]) + ".ckpt")
comet_logger.experiment.log_asset_folder(checkpoint_folder)